## Accumulate and Decay <sup>14</sup>C
<br>
  
#### Syntax
`conc14C = C14AnnualFxn(conc0, CosmoPars)` <br>
#### Input 
`conc0` : vector of size len(CRNz) with spacing dz containing the initial concentration (at g-1)<br>
`CosmoPars` : dictionary of cosmogenic nuclide accumulation/decay parameters.<br>
#### Variables Used
##### Cosmo Pars
`CRNz` : depth vector of len(max_depths / dz) <br>
`scaling_factor` : CN scaling factor (e.g. Stone 2000) <br> 
`rock_rho` :  = bulk rock density (g cm-3) <br>
`mu` : att length / density <br>
`SLHLC14` : SLHL production rate for 14C <br>
`C14 thalf` : C14 half life half-life <br>
`C14_decay` = decay constant for C14 <br>
`C14 f_tot` = total muon flux <br>
`C14 A1` = pre-exponential term for fast muon production <br>
`C14 A2` = pre-exponential term for fast muon production <br>
`C14 L1` = attenuation length for fast muon production <br>
`C14 L2` = attenuation length for fast muon production <br>   
#### Output
`conc14C` : vector of size len(CRNz) with spacing dz containing concentrations after this additional year (at g-1).<br>
#### Notes
**Date of Creation:** 1. Juli 2021 <br>
**Author:** Donovan Dennis <br>
**Update:** <br>

In [1]:
def C14AnnualFxn(conc0, CosmoPars):    
    # import the relevant cosmo acc/decay parameters
    
    CRNz = CosmoPars['CRNz']
    scaling_factor = CosmoPars['scaling_factor']
    mu = CosmoPars['mu']
    rock_rho = CosmoPars['rock_rho']
    SLHLC14 = CosmoPars['SLHL_C14']
    thalf = CosmoPars['C14 thalf']
    C14_decay = CosmoPars['C14 decay'] 
    f_tot = CosmoPars['C14 f_tot'] 
    A1 = CosmoPars['C14 A1']
    A2 = CosmoPars['C14 A2']
    L1 = CosmoPars['L1 C14']
    L2 = CosmoPars['L2 C14']   
    
    C14_P0 = SLHLC14 * scaling_factor   # production rate atoms g-1 yr-1
    
    # loop to determine fast muon concentration at depth CRNz[i]
    muons = [f_tot * (A1*np.e**((-rock_rho * CRNz[i])/L1) + A2*np.e**((-rock_rho*CRNz[i])/L2)) for i in range(len(CRNz))]  
    # and loop again for spallogenic concentration at depth CRNz[i]
    spallation = [C14_P0 * np.e**(-mu * CRNz[i]) for i in range(len(CRNz))]
    # grids match so can directly add concentrations to each other
    production_tot_14 = [spallation[i] + muons[i] for i in range(len(CRNz))]
    
    # return the total concentration after C-14 decay
    conc14C = [conc0[i]*np.e**(-C14_decay) + (production_tot_14[i] / (C14_decay + 0) * (1 - np.e**(-(C14_decay + 0) * 1))) for i in range(len(CRNz))]  
    
    return conc14C